In [1]:
from pathlib import Path
import duckdb
import dlt
from defi_ds.data import coingecko_prices
rootpath = Path.cwd()
from datetime import datetime
duckdb_destination = rootpath / "data/coingecko_pipeline.duckdb"


In [4]:
c = []
for data in coingecko_prices("bitcoin"):
    c.append(data)

In [6]:
c

[{'timestamp': 1742979921981, 'price': 87983.70142460795},
 {'timestamp': 1742983285788, 'price': 87922.27667040464},
 {'timestamp': 1742987095270, 'price': 88240.46256324385},
 {'timestamp': 1742990668151, 'price': 87998.8047039292},
 {'timestamp': 1742994288961, 'price': 87795.45224485123},
 {'timestamp': 1742997830683, 'price': 87219.71294548987},
 {'timestamp': 1743001491003, 'price': 86823.44172743564},
 {'timestamp': 1743004871211, 'price': 86745.53174766526},
 {'timestamp': 1743008675693, 'price': 86454.04031914235},
 {'timestamp': 1743012280575, 'price': 86566.57610290994},
 {'timestamp': 1743015893403, 'price': 86217.63942503363},
 {'timestamp': 1743019490699, 'price': 86625.17449589507},
 {'timestamp': 1743023070530, 'price': 87199.42852219005},
 {'timestamp': 1743026581708, 'price': 87150.9659589762},
 {'timestamp': 1743030480483, 'price': 86817.98796198177},
 {'timestamp': 1743033881339, 'price': 86913.68578612641},
 {'timestamp': 1743037479772, 'price': 87277.52244284363},

In [5]:
@dlt.resource(
    columns={
        "timestamp": {"data_type": "timestamp", "timezone": "bool"},
        "price": {"data_type": "decimal", "precision": 3},
    }
)
def coingecko_prices_mock(
    coin_id: str,
    vs_currency: str = "usd",
    days: Optional[int] = 90,
):
    """Resource for CoinGecko price data only."""

    data = c

    # if "prices" in data:
    for timestamp_ms, price in data:
        yield {
            "timestamp": timestamp_ms,
            "price": price,
        }

[{'timestamp': 1742979921981, 'price': 87983.70142460795},
 {'timestamp': 1742983285788, 'price': 87922.27667040464},
 {'timestamp': 1742987095270, 'price': 88240.46256324385},
 {'timestamp': 1742990668151, 'price': 87998.8047039292},
 {'timestamp': 1742994288961, 'price': 87795.45224485123},
 {'timestamp': 1742997830683, 'price': 87219.71294548987},
 {'timestamp': 1743001491003, 'price': 86823.44172743564},
 {'timestamp': 1743004871211, 'price': 86745.53174766526},
 {'timestamp': 1743008675693, 'price': 86454.04031914235},
 {'timestamp': 1743012280575, 'price': 86566.57610290994},
 {'timestamp': 1743015893403, 'price': 86217.63942503363},
 {'timestamp': 1743019490699, 'price': 86625.17449589507},
 {'timestamp': 1743023070530, 'price': 87199.42852219005},
 {'timestamp': 1743026581708, 'price': 87150.9659589762},
 {'timestamp': 1743030480483, 'price': 86817.98796198177},
 {'timestamp': 1743033881339, 'price': 86913.68578612641},
 {'timestamp': 1743037479772, 'price': 87277.52244284363},

In [2]:
# Create a pipeline
pipeline = dlt.pipeline(
    pipeline_name="coingecko_pipeline",
    destination=dlt.destinations.duckdb(str(duckdb_destination)),
    dataset_name="crypto_data"
)

# Use any of the resources
load_info = pipeline.run(
    coingecko_prices("bitcoin"),
    table_name="bitcoin_prices",
    write_disposition="replace"
)


PipelineStepFailed: Pipeline execution failed at `step=extract` when processing package with `load_id=1750751516.8630836` with exception:

<class 'dlt.extract.exceptions.ResourceExtractionError'>
In processing pipe `coingecko_prices`: extraction of resource `coingecko_prices` in `generator` `coingecko_prices` caused an exception: 400 Client Error: Bad Request for url: https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=90

In [4]:
sql = f"""
SELECT * FROM information_schema.tables
"""
conn = duckdb.connect(str(duckdb_destination))
conn.execute(sql).fetchdf()


,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,coingecko_pipeline,crypto_data,bitcoin_prices,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,coingecko_pipeline,crypto_data,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,coingecko_pipeline,crypto_data,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,coingecko_pipeline,crypto_data,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [4]:
sql = f"""
SELECT * FROM crypto_data.bitcoin_prices
"""
conn = duckdb.connect(str(duckdb_destination))
conn.execute(sql).fetchdf()


,timestamp,price,price__v_double,_dlt_load_id,_dlt_id
0,2025-03-26 22:04:37.452000+07:00,88159,NaN,1750751179.7374952,PBFk0QEIS1oJCA
1,2025-03-26 23:05:21.981000+07:00,87984,NaN,1750751179.7374952,YSswp+t4br3Wrw
2,2025-03-27 00:01:25.788000+07:00,87922,NaN,1750751179.7374952,JV5piZrCF7Rnkg
3,2025-03-27 01:04:55.270000+07:00,88240,NaN,1750751179.7374952,jz6+mXHk2kBsgw
4,2025-03-27 02:04:28.151000+07:00,87999,NaN,1750751179.7374952,oeIehGLHoOriIQ
...,...,...,...,...,...
2155,2025-06-24 18:03:24.192000+07:00,105013,NaN,1750751179.7374952,RC5AdKFYnFzWDA
2156,2025-06-24 19:03:09.491000+07:00,104920,NaN,1750751179.7374952,GvNhnf96pYDNGw
2157,2025-06-24 20:03:25.365000+07:00,105447,NaN,1750751179.7374952,/GYsLYasFmYFTQ
2158,2025-06-24 21:03:36.878000+07:00,105730,NaN,1750751179.7374952,eQPdSFrv+djvxQ
